 <em> <font color=#AA0CAA>Primer paso: instalamos las librerias de google cloud y vision</font> </em>

In [1]:
!pip install google-cloud-storage


In [2]:
!pip install google-cloud

In [3]:
!pip install google-cloud-vision

In [4]:
!pip install pandas

 <em> <font color=#AA0CAA>Segundo paso: importamos google cloud y vision</font> </em>

In [5]:
from __future__ import print_function
import os
from google.cloud import storage
import google.cloud.storage
import json
import argparse
import io
import sys
import logging
import pandas as pd
from google.cloud import vision
from google.cloud import storage

 <em> <font color=#AA0CAA>Explicacion: Aqui lo que hacemos es cargar nuestras credenciales para poder acceder al storage y activar la cuenta en vision</font> </em>

In [6]:
PATH= os.path.join(os.getcwd(),'ueconcurrente-8be47ff1066b.json')
print('\n',PATH)
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=PATH
storage_client= storage.Client(PATH)
storage_client
bucket=storage_client.get_bucket('nomina_01')
filename=list(bucket.list_blobs(prefix=''))
#aqui verifico que la nomina se encuentre dentro de mi bucket
for name in filename:
    print(name.name)



 /Users/adilemdobras/ueconcurrente-8be47ff1066b.json
NominaConcurrente.jpg


In [7]:
#Explicacion: metodo que verifico los digitos y solo los valores numericos en la nomina, me devuelve un boolean
def if_integer(string):
    return any(chr.isdigit() for chr in string)  

In [8]:
#Explicacion: cargo la imagen desde el url del bucket y lo transporto a vision
image_uri = 'gs://nomina_01/NominaConcurrente.jpg'
client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri
response = client.text_detection(image=image)
texts = response.text_annotations
print('Texts:')

#creo un frame csv para poder ver el texto obtenido
tabla =pd.DataFrame(columns=['Texto encontrado'])
#acrivo vision para buscar los textos, recorro el json
for text in texts:
    result='\n"{}"'.format(text.description)
    if_integer(result)
   
    print(result)

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                 for vertex in text.bounding_poly.vertices])
  
    tabla = tabla.append(dict(
                Texto_encontrado = text.description
            ),ignore_index=True)
       
    tabla.to_csv('nominas.csv')

Texts:

"Ejemplo, s.l
Trabajador Eienmplo
EMPRESA
DOMICILIO
TRABAJADOR
NF
CATEGORÍA
GRUPO COTIZACIÓN
CIF
Ayudante
4.
Periodo liquidación
I. DEVENGOS
1. Percepciones salariales
Sdario base
De 1
Septiembre
Septiembre
N.º días
30
30
TOTALES
1.100,00
Complementos salariales
Flus actividad
200,00
Hors extraordinaias
Horas complerentarias
Gratificadones extraor dir
Sario en especie
2. Percepciones no salaria
Indermizaciones o Suplid
Prestaciones e indemmizaciones de la Seguridad Socia
Ctras percepciones no salarides
A TOTAL DEVENGADO
1.300,00 €
II. DEDUCCIONES
1. Aportaciones del trabajador a las cotizadiones a la S.S v recaudación conjunta
Contingencias comunes
Desempleo
Formación Profesiona
Horas extraordinarias Normales
Horas extraordinarias de Fuerza Mayor
TOTAL APORTACIONE
Porcentaje
4,70%
1,60%
0,10%
4,70%
2,00%
1.483,33€
1.483,33€
1.483,33€
69,72
23,73
1,48
94,93
2. RF
3. Anticipos
4. Valor de los productos recibidos en especie
5. Ctras deducciones
1.300,00 €
5,00%
65
B. TOTAL A DEDUC

 <em> <font color=#AA0CAA>Explicacion: Aqui lo que hago es guardar en el csv el tipo de dato que es el texto encontrado para despues poder filtrar los digitos</font> </em>

In [9]:
datos=pd.read_csv('nominas.csv')
df=pd.DataFrame(datos)
alto=df.loc[0,'Texto_encontrado']
tabla2=pd.DataFrame(columns=['Texto','Tipo_Texto'])
#tabla2 =pd.DataFrame(columns=['Tipo_Texto_encontrado'])
#tabla2['Tipo_Texto'] = tipo
#tabla2['Texto'] = texto

for i in range(0,len(df)):
    tipo=df.loc[i,'Texto_encontrado']
    texto=if_integer(df.loc[i,'Texto_encontrado'])
    tabla2.loc[i]=[tipo,texto]
    tabla2.to_csv('nominas2.csv')
print(tabla2)
 

                                                 Texto Tipo_Texto
0    Ejemplo, s.l\nTrabajador Eienmplo\nEMPRESA\nDO...       True
1                                             Ejemplo,      False
2                                                  s.l      False
3                                           Trabajador      False
4                                             Eienmplo      False
..                                                 ...        ...
244                                          retención      False
245                                                 de      False
246                                                IRF      False
247                                           1.300,00       True
248                                                  €      False

[249 rows x 2 columns]
